In [1]:
import numpy as np
import pandas as pd
from pyscheduling.FS import FmCmax, FlowShop
import pickle
import time
import matplotlib.pyplot as plt
import numpy as np
import random

## Makespan

In [2]:
def compute_makespan(schedule, p):
    _, m = p.shape
    n = len(schedule)
    c = [[0]*m for i in range(n)]
    for i in range(n):
        for j in range(m):
            if i == 0 and j == 0:
                c[i][j] = p[schedule[i]][j]
            elif i == 0:
                c[i][j] = c[i][j-1] + p[schedule[i]][j]
            elif j == 0:
                c[i][j] = c[i-1][j] + p[schedule[i]][j]
            else:
                c[i][j] = max(c[i][j-1], c[i-1][j]) + p[schedule[i]][j]
    return c[n-1][m-1]

## Heuristics (to be used for initialisation)

In [3]:
def heuristique_Ham(M):
    m = M.shape[1]

    P1 = np.sum(M[:,:m//2], axis=1) # somme des durées sur la première machine
    P2 = np.sum(M[:,m//2:], axis=1) # somme des durées sur la deuxième machine
    P2_P1 = P2 - P1 # différence entre les deux sommes
    ordre = np.flip(np.argsort(P2_P1)) # tri par ordre décroissant

    # Première solution
    ordre_1 = ordre
    Cmax1 = compute_makespan(ordre_1 , M) # makespan de la première solution

    # Deuxième solution
    indice_positif = ordre[P2_P1[ordre] >= 0] # indices des tâches avec un indice positif ou nul
    indice_negatif = ordre[P2_P1[ordre] < 0] # indices des tâches avec un indice négatif

    indice_positif = [indice_positif[i] for i in np.argsort(P1[indice_positif])] # tri croissant selon P1
    indice_negatif = [indice_negatif[i] for i in np.flip(np.argsort(P2[indice_negatif]))] # tri décroissant selon P2
   
    ordre_2 = [int(i) for i in np.concatenate((indice_positif, indice_negatif))]
    Cmax2 = compute_makespan(ordre_2 , M) # makespan de la deuxième solution

    if (Cmax1 > Cmax2):
        return ordre_1, Cmax1
    else:
        return ordre_2, Cmax2

In [80]:
a,b=heuristique_NEH(processing_times)
a,b

([8, 3, 6, 0, 1, 2, 4, 7, 5, 9], 1107)

## Iterated Local Search proposed by dong et al (2009)
##### (https://sci-hub.ru/10.1016/j.cor.2008.04.001)

### utilities for the dong ils

In [127]:
def perform_insert(schedule, i, j):
    """
    Performs an INSERT move on the given permutation by moving the job at
    position i to position j.
    
     - i: the position of the job to be moved.
     - j: the position where the job should be inserted.
    """
    schedule=list(schedule)
    new_schedule = schedule.copy()
    job = new_schedule.pop(i)
    new_schedule.insert(j, job)
    return new_schedule

def pairwise_permute_n(schedule, n_perturb):
    """
    Performs n random pairwise permutations between elements in a list.
    
    """
    new_schedule = schedule.copy()
    for i in range(n_perturb):
        idx1, idx2 = random.sample(range(len(new_schedule)), 2)
        new_schedule[idx1], new_schedule[idx2] = new_schedule[idx2], new_schedule[idx1]
    return new_schedule

# job here is the index of job in the schedule not the job 
# saying schedule [3,0,2,1]  job=1 is 3, job =2 is 0 ...
def completeion_time_of_job(machine,job,schedule,processing_times):
    if(job==1):
        cmp=0
        for r in range(machine):
            cmp=cmp+processing_times[schedule[0]][r]
        return cmp
    elif(machine==1):
        cmp=0
        for r in range(job):
            cmp=cmp+processing_times[schedule[r]][0]
        return cmp
    else:
        e1=completeion_time_of_job(machine-1,job,schedule,processing_times)
        e2=completeion_time_of_job(machine,job-1,schedule,processing_times)
        return processing_times[job-1][machine-1]+max(e1,e2)

### ILS metaheuristic

In [128]:
def dong_et_al_ils(schedule,processing_times,max_iter,n_perturb):
    n_jobs , n_machines = processing_times.shape
    cnt=0
    schedule=list(schedule)
    schedule_best=schedule;
    
    for itr in range(max_iter):
        for j in range(n_jobs):
            
            #selecting the k
            k=0
            obj=completeion_time_of_job(n_machines,1,schedule,processing_times)
            for e in range(n_jobs-1):
                if(completeion_time_of_job(n_machines,e+1,schedule,processing_times)<obj):
                    obj=completeion_time_of_job(n_machines,e+1,schedule,processing_times)
                    k=e;
            # do insertions:
            obj=compute_makespan(schedule, processing_times)
            schedule_fat7a=schedule
            schedule_optim=schedule
            for e in range(n_jobs):
                new_schedule=perform_insert(schedule, k, e)
                if(compute_makespan(new_schedule, processing_times)<obj):
                    schedule_optim=new_schedule

            if(compute_makespan(schedule_optim, processing_times)<compute_makespan(schedule, processing_times)):
                schedule=schedule_optim
                cnt=0
            else:
                cnt=cnt+1

            if(compute_makespan(schedule, processing_times)<compute_makespan(schedule_best, processing_times)):
                schedule_best=schedule

            if(cnt>=n_jobs):
                ss=pairwise_permute_n(schedule_best, pert_times)
                if(compute_makespan(ss, processing_times)<compute_makespan(schedule_best, processing_times)):
                    schedule_best=ss
                cnt=0
    return schedule_best,compute_makespan(schedule_best, processing_times)

## TESTS

In [119]:
instance=FmCmax.FmCmax_Instance.read_txt("../TP02-Heuristiques/data/random_instance.txt")
processing_times = np.array(instance.P)
schedule,b=heuristique_Ham(M)
upper_bound,schedule,compute_makespan(schedule, M)

In [134]:
f =  open("../TP02-Heuristiques/data/Taillard.pkl", "rb")
taillard = pickle.load(f)
i=0;
max_iter=200
nb_perturb=8
M = np.array(taillard[i]["P"]).transpose()
upper_bound = taillard[i]["ub"]
schedule,b=heuristique_Ham(M)
processing_times=M
schedule=list(schedule)
upper_bound,schedule,compute_makespan(schedule, M)

for i in range(10):
    processing_times = np.array(taillard[i]["P"]).transpose()
    upper_bound = taillard[i]["ub"]
    init_schedule,init_obj=heuristique_Ham(processing_times)
    start_time = time.time()
    schedule,obj=dong_et_al_ils(init_schedule,processing_times,max_iter,nb_perturb)
    end_time = time.time()
    print("-------------- Instance ",i+1,"------------------")
    print("instance",i+1,":","deviation: ",100*(compute_makespan(schedule,processing_times)-upper_bound)/upper_bound,"% ")
    print("improved compared to HAM : ",100*(init_obj-obj)/(init_obj-upper_bound))
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")
    print("-------------------------------------------------\n\n")

-------------- Instance  1 ------------------
instance 1 : deviation:  5.555555555555555 % 
improved compared to HAM :  55.34591194968554
Elapsed time: 239.15074634552002 seconds
-------------------------------------------------


-------------- Instance  2 ------------------
instance 2 : deviation:  1.7660044150110374 % 
improved compared to HAM :  81.3953488372093
Elapsed time: 225.80355095863342 seconds
-------------------------------------------------


-------------- Instance  3 ------------------
instance 3 : deviation:  9.805735430157261 % 
improved compared to HAM :  67.48466257668711
Elapsed time: 224.15408635139465 seconds
-------------------------------------------------


-------------- Instance  4 ------------------
instance 4 : deviation:  10.05413766434648 % 
improved compared to HAM :  48.616600790513836
Elapsed time: 238.27361416816711 seconds
-------------------------------------------------


-------------- Instance  5 ------------------
instance 5 : deviation:  6.95

In [135]:
f =  open("../TP02-Heuristiques/data/Taillard.pkl", "rb")
taillard = pickle.load(f)
i=0;
max_iter=20
nb_perturb=4
M = np.array(taillard[i]["P"]).transpose()
upper_bound = taillard[i]["ub"]
schedule,b=heuristique_Ham(M)
processing_times=M
schedule=list(schedule)
upper_bound,schedule,compute_makespan(schedule, M)

for i in range(10):
    processing_times = np.array(taillard[i]["P"]).transpose()
    upper_bound = taillard[i]["ub"]
    init_schedule,init_obj=heuristique_Ham(processing_times)
    start_time = time.time()
    schedule,obj=dong_et_al_ils(init_schedule,processing_times,max_iter,nb_perturb)
    end_time = time.time()
    print("-------------- Instance ",i+1,"------------------")
    print("instance",i+1,":","deviation: ",100*(compute_makespan(schedule,processing_times)-upper_bound)/upper_bound,"% ")
    print("improved compared to HAM : ",100*(init_obj-obj)/(init_obj-upper_bound))
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")
    print("-------------------------------------------------\n\n")

-------------- Instance  1 ------------------
instance 1 : deviation:  7.589984350547731 % 
improved compared to HAM :  38.9937106918239
Elapsed time: 23.82146906852722 seconds
-------------------------------------------------


-------------- Instance  2 ------------------
instance 2 : deviation:  3.164091243561442 % 
improved compared to HAM :  66.66666666666667
Elapsed time: 22.645424842834473 seconds
-------------------------------------------------


-------------- Instance  3 ------------------
instance 3 : deviation:  19.42645698427382 % 
improved compared to HAM :  35.58282208588957
Elapsed time: 23.12198042869568 seconds
-------------------------------------------------


-------------- Instance  4 ------------------
instance 4 : deviation:  18.561484918793504 % 
improved compared to HAM :  5.138339920948616
Elapsed time: 23.51092505455017 seconds
-------------------------------------------------


-------------- Instance  5 ------------------
instance 5 : deviation:  8.737864

In [136]:
f =  open("../TP02-Heuristiques/data/Taillard.pkl", "rb")
taillard = pickle.load(f)
i=0;
max_iter=20
nb_perturb=8
M = np.array(taillard[i]["P"]).transpose()
upper_bound = taillard[i]["ub"]
schedule,b=heuristique_Ham(M)
processing_times=M
schedule=list(schedule)
upper_bound,schedule,compute_makespan(schedule, M)

for i in range(10):
    processing_times = np.array(taillard[i]["P"]).transpose()
    upper_bound = taillard[i]["ub"]
    init_schedule,init_obj=heuristique_Ham(processing_times)
    start_time = time.time()
    schedule,obj=dong_et_al_ils(init_schedule,processing_times,max_iter,nb_perturb)
    end_time = time.time()
    print("-------------- Instance ",i+1,"------------------")
    print("instance",i+1,":","deviation: ",100*(compute_makespan(schedule,processing_times)-upper_bound)/upper_bound,"% ")
    print("improved compared to HAM : ",100*(init_obj-obj)/(init_obj-upper_bound))
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")
    print("-------------------------------------------------\n\n")

-------------- Instance  1 ------------------
instance 1 : deviation:  8.294209702660407 % 
improved compared to HAM :  33.333333333333336
Elapsed time: 24.483702182769775 seconds
-------------------------------------------------


-------------- Instance  2 ------------------
instance 2 : deviation:  5.003679175864606 % 
improved compared to HAM :  47.286821705426355
Elapsed time: 23.058905601501465 seconds
-------------------------------------------------


-------------- Instance  3 ------------------
instance 3 : deviation:  24.514338575393154 % 
improved compared to HAM :  18.711656441717793
Elapsed time: 22.749699592590332 seconds
-------------------------------------------------


-------------- Instance  4 ------------------
instance 4 : deviation:  13.45707656612529 % 
improved compared to HAM :  31.225296442687746
Elapsed time: 24.356608629226685 seconds
-------------------------------------------------


-------------- Instance  5 ------------------
instance 5 : deviation:  

In [114]:
max_iter=400
n_jobs=20
n_machines=5
schedule_best=schedule;
cnt=0;
pert_times=6
for i in range(max_iter):
    for j in range(n_jobs):
        
        #selecting the k
        k=0
        obj=completeion_time_of_job(n_machines,1,schedule,processing_times)
        for e in range(n_jobs-1):
            if(completeion_time_of_job(n_machines,e+1,schedule,processing_times)<obj):
                obj=completeion_time_of_job(n_machines,e+1,schedule,processing_times)
                k=e;
        
        # do insertions:
        obj=compute_makespan(schedule, processing_times)
        schedule_fat7a=schedule
        schedule_optim=schedule
        for e in range(n_jobs):
            new_schedule=perform_insert(schedule, k, e)
            if(compute_makespan(new_schedule, processing_times)<obj):
                schedule_optim=new_schedule
        
        if(compute_makespan(schedule_optim, processing_times)<compute_makespan(schedule, processing_times)):
            schedule=schedule_optim
            cnt=0
        else:
            cnt=cnt+1
        
        if(compute_makespan(schedule, processing_times)<compute_makespan(schedule_best, processing_times)):
            schedule_best=schedule
        
        if(cnt>=n_jobs):
            ss=pairwise_permute_n(schedule_best, pert_times)
            if(compute_makespan(ss, processing_times)<compute_makespan(schedule_best, processing_times)):
                schedule_best=ss
            cnt=0
compute_makespan(schedule_best, processing_times),schedule_best

(1297, [14, 3, 2, 15, 12, 0, 18, 8, 7, 16, 4, 5, 11, 17, 6, 10, 1, 13, 9, 19])

## Guided Local Search 

### Tabu Search

### Pseudo code:
1-Initialize the tabu list and set the initial solution as the current solution.
2-Define a neighborhood structure that generates candidate solutions by swapping adjacent jobs in the current solution.
3-Evaluate the quality of the initial solution using an objective function that measures the total processing time on all machines.
4-Repeat until a stopping criterion is met:
    a. Generate a set of candidate solutions by applying the neighborhood structure to the current solution.
    b. Remove any candidate solutions that are in the tabu list.
    c. Evaluate the remaining candidate solutions using the objective function.
    d. Select the best candidate solution as the new current solution.
    e. Update the tabu list with the current solution.
    f. If the new current solution is better than the best solution found so far, update the best solution.
5Return the best solution found.

In [40]:
def tabu_search(init_schedule,processing_times, neighbors_size=10, max_iter=1000, tabu_size=10):
    
    n_jobs , n_machines = processing_times.shape
    current_schedule = init_schedule.copy()
    best_schedule = init_schedule.copy()
    
    tabu_list = []
    
    iter = 0
    for _ in range(max_iter):
        
        # Générer un ensemble de solutions candidates
        candidates = []
        for i in range(neighbors_size):
            candidate = current_schedule.copy()
            j = random.randint(0, n_jobs-2)
            k = random.randint(j+1, n_jobs-1)
            candidate[j:k+1] = reversed(candidate[j:k+1])
            
            # Supprimer le candidat s'il figure dans la liste taboue
            if(candidate not in tabu_list):
                candidates.append(candidate)
        
        # Évaluer les solutions candidates restantes
        candidate_scores = [compute_makespan(candidate, processing_times) for candidate in candidates]
        
        # Sélectionner la meilleure solution candidate comme nouvelle solution actuelle
        best_candidate_idx = min(range(len(candidate_scores)), key=lambda i: candidate_scores[i])
        current_schedule = candidates[best_candidate_idx]
        
        # Ajouter la solution actuelle à la liste taboue
        tabu_list.append(current_schedule.copy())
        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)
            
        # Vérifier si le meilleur candidat est meilleur que la solution actuelle
        if candidate_scores[best_candidate_idx] < compute_makespan(best_schedule, processing_times):
            best_schedule = current_schedule.copy()

    return best_schedule

## Tests

In [37]:
instance=FmCmax.FmCmax_Instance.read_txt("../TP02-Heuristiques/data/random_instance.txt")
processing_times = np.array(instance.P)
# schedule,b=heuristique_Ham(M)
# upper_bound,schedule,compute_makespan(schedule, M)
processing_times

array([[71, 79, 85, 82, 83],
       [84, 71, 66, 68, 81],
       [78, 81, 75, 72, 87],
       [78, 75, 66, 72, 88],
       [72, 88, 83, 85, 88],
       [86, 88, 79, 82, 78],
       [75, 66, 86, 78, 78],
       [80, 79, 66, 83, 78],
       [73, 73, 67, 77, 71],
       [80, 77, 83, 78, 67]])

In [38]:
initial_solution = list(range(10))
random.shuffle(initial_solution)
compute_makespan(tabu_search(initial_solution, processing_times,neighbors_size=20, max_iter=5000, tabu_size=100),processing_times)

1102

### taillard

In [50]:
f =  open("../TP02-Heuristiques/data/Taillard.pkl", "rb")
taillard = pickle.load(f)
i=0;
max_iter=5000
tabu_size=100
neighbors_size=20
for i in range(10):
    processing_times = np.array(taillard[i]["P"]).transpose()
    upper_bound = taillard[i]["ub"]
    init_schedule=list(range(20));
    random.shuffle(init_schedule)
    init_obj=compute_makespan(init_schedule,processing_times)
    start_time = time.time()
    schedule=tabu_search(init_schedule, processing_times,neighbors_size, max_iter, tabu_size)
    end_time = time.time()
    obj=compute_makespan(schedule,processing_times)
    print("-------------- Instance ",i+1,"------------------")
    print("instance",i+1,":","deviation: ",100*(obj-upper_bound)/upper_bound,"% ")
    print("improved compared to init solution: ",100*(init_obj-obj)/(init_obj-upper_bound))
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")
    print("-------------------------------------------------\n\n")

-------------- Instance  1 ------------------
instance 1 : deviation:  0.0 % 
improved compared to init solution:  100.0
Elapsed time: 5.7258055210113525 seconds
-------------------------------------------------


-------------- Instance  2 ------------------
instance 2 : deviation:  0.07358351729212656 % 
improved compared to init solution:  99.46236559139786
Elapsed time: 5.869294881820679 seconds
-------------------------------------------------


-------------- Instance  3 ------------------
instance 3 : deviation:  0.7400555041628122 % 
improved compared to init solution:  97.4921630094044
Elapsed time: 5.828875541687012 seconds
-------------------------------------------------


-------------- Instance  4 ------------------
instance 4 : deviation:  0.6187161639597835 % 
improved compared to init solution:  97.2508591065292
Elapsed time: 5.937375783920288 seconds
-------------------------------------------------


-------------- Instance  5 ------------------
instance 5 : deviatio

In [52]:
f =  open("../TP02-Heuristiques/data/Taillard.pkl", "rb")
taillard = pickle.load(f)
i=0;
max_iter=5000
tabu_size=10
neighbors_size=40
for i in range(10):
    processing_times = np.array(taillard[i]["P"]).transpose()
    upper_bound = taillard[i]["ub"]
    init_schedule=list(range(20));
    random.shuffle(init_schedule)
    init_obj=compute_makespan(init_schedule,processing_times)
    start_time = time.time()
    schedule=tabu_search(init_schedule, processing_times,neighbors_size, max_iter, tabu_size)
    end_time = time.time()
    obj=compute_makespan(schedule,processing_times)
    print("-------------- Instance ",i+1,"------------------")
    print("instance",i+1,":","deviation: ",100*(obj-upper_bound)/upper_bound,"% ")
    print("improved compared to init solution: ",100*(init_obj-obj)/(init_obj-upper_bound))
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")
    print("-------------------------------------------------\n\n")

-------------- Instance  1 ------------------
instance 1 : deviation:  0.0 % 
improved compared to init solution:  100.0
Elapsed time: 12.948039770126343 seconds
-------------------------------------------------


-------------- Instance  2 ------------------
instance 2 : deviation:  0.515084621044886 % 
improved compared to init solution:  92.78350515463917
Elapsed time: 13.03857684135437 seconds
-------------------------------------------------


-------------- Instance  3 ------------------
instance 3 : deviation:  0.0 % 
improved compared to init solution:  100.0
Elapsed time: 13.531136512756348 seconds
-------------------------------------------------


-------------- Instance  4 ------------------
instance 4 : deviation:  0.46403712296983757 % 
improved compared to init solution:  97.63779527559055
Elapsed time: 12.997552394866943 seconds
-------------------------------------------------


-------------- Instance  5 ------------------
instance 5 : deviation:  -0.08090614886731391

In [54]:
f =  open("../TP02-Heuristiques/data/Taillard.pkl", "rb")
taillard = pickle.load(f)
i=0;
max_iter=5000
tabu_size=10
neighbors_size=40
for i in range(1):
    i=4
    processing_times = np.array(taillard[i]["P"]).transpose()
    upper_bound = taillard[i]["ub"]
    init_schedule=list(range(20));
    random.shuffle(init_schedule)
    init_obj=compute_makespan(init_schedule,processing_times)
    start_time = time.time()
    schedule=tabu_search(init_schedule, processing_times,neighbors_size, max_iter, tabu_size)
    end_time = time.time()
    obj=compute_makespan(schedule,processing_times)
    print("-------------- Instance ",i+1,"------------------")
    print(schedule,obj,upper_bound)
    print("instance",i+1,":","deviation: ",100*(obj-upper_bound)/upper_bound,"% ")
    print("improved compared to init solution: ",100*(init_obj-obj)/(init_obj-upper_bound))
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")
    print("-------------------------------------------------\n\n")

-------------- Instance  5 ------------------
[11, 18, 2, 8, 3, 9, 16, 15, 1, 12, 4, 14, 5, 10, 13, 6, 17, 0, 19, 7] 1235 1236
instance 5 : deviation:  -0.08090614886731391 % 
improved compared to init solution:  100.47846889952153
Elapsed time: 11.21304178237915 seconds
-------------------------------------------------


